<a href="https://colab.research.google.com/github/muhajirakbarhsb/NLP_class_2023/blob/main/Week_Class_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Week Class 5

In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data_clean_all.csv')
df

,at,content,score,userName,contentp,contentp_clean
0,2023-10-06 10:07:21,"makin sini makin parah , Lola, mahal",1,MIFTAHUDDIN JUHDI,makin sini makin parah lola mahal,makin sini makin parah lambat mahal
1,2023-10-06 07:27:57,Makin hari paket nya makin mahal,1,Fandi Rahmadi,makin hari paket nya makin mahal,makin hari paket nya makin mahal
2,2023-10-06 06:12:55,"Semua saran sudah kami lakukan, hasilnya tetap...",1,Jimmy Jimmy,semua saran sudah kami lakukan hasilnya tetap ...,semua saran kami laku hasil tetap lambat sisa ...
3,2023-10-06 04:12:21,Aplikasi yang baik,5,Princess Khan,aplikasi yang baik,aplikasi baik
4,2023-10-05 15:41:08,"Siapa yang menyesal beli modem ini,🙋🙋🙋 aku aku...",1,PETER 65,siapa yang menyesal beli modem ini aku aku aku...,siapa sesal beli modem aku aku aku paket nya m...
...,...,...,...,...,...,...
3183,2023-01-01 08:28:28,Internet jelek,1,Rafael,internet jelek,internet jelek
3184,2023-01-01 07:54:02,Pilihan Menu pembelian paket nya gak ada.. gan...,5,Rahmat Samsudin,pilihan menu pembelian paket nya gak ada gangg...,pilih menu beli paket nya ada ganggu
3185,2023-01-01 06:10:55,Sekarang gak bisa beli kuota 100gb... cuma ada...,1,aby habibie,sekarang gak bisa beli kuota gb cuma ada gb gb...,sekarang bisa beli kuota gb cuma gb gb buat ap...
3186,2023-01-01 02:48:24,Saya sudah pakai orbit hampir 1thn tapi kemari...,5,Azkayrah Zahwa,saya sudah pakai orbit hampir thn tapi kemarin...,pakai orbit hampir thn kemarin mau sambung hp ...


In [4]:
df['sentiment_first_scheme'] = df['score'].apply(lambda x: 'negative' if x in [1, 2] else 'positive')

# Sentiment analysis based on the second scheme
df['sentiment_second_scheme'] = df['score'].apply(lambda x: 'negative' if x in [1, 2] else ('neutral' if x == 3 else 'positive'))

In [7]:
df.sentiment_second_scheme.value_counts()

negative    1717
positive    1243
neutral      228
Name: sentiment_second_scheme, dtype: int64

In [8]:
df.sentiment_first_scheme.value_counts()

negative    1717
positive    1471
Name: sentiment_first_scheme, dtype: int64

### Classification Process

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [16]:
df = df[['contentp_clean', 'sentiment_first_scheme', 'score']]
missing_values = df['contentp_clean'].isna().sum()
if missing_values > 0:
    df['contentp_clean'].fillna('', inplace=True)


In [17]:
X = df['contentp_clean']  # Text data
y_binary = (df['sentiment_first_scheme'] == 'positive').astype(int)  # Binary labels (1 for positive, 0 for negative)
y_multi = df['score']  # Multi-class labels (original star ratings)

X_train, X_test, y_binary_train, y_binary_test, y_multi_train, y_multi_test = train_test_split(
    X, y_binary, y_multi, test_size=0.2, random_state=42)

In [18]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [20]:
X_train_tfidf

<2550x2880 sparse matrix of type '<class 'numpy.float64'>'
	with 24736 stored elements in Compressed Sparse Row format>

In [21]:
X_train_tfidf_dense = X_train_tfidf.toarray()

# Print the TF-IDF representation
print("TF-IDF representation of X_train:")
print(X_train_tfidf_dense)

TF-IDF representation of X_train:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [22]:
binary_classifier = LogisticRegression()
binary_classifier.fit(X_train_tfidf, y_binary_train)
binary_predictions = binary_classifier.predict(X_test_tfidf)

In [23]:
print("Binary Classification Report:")
print(classification_report(y_binary_test, binary_predictions))

Binary Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.92      0.85       330
           1       0.89      0.73      0.81       308

    accuracy                           0.83       638
   macro avg       0.84      0.83      0.83       638
weighted avg       0.84      0.83      0.83       638



In [24]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [25]:
# Binary Classification with Random Forest and GridSearchCV
rf_classifier = RandomForestClassifier()
param_grid_rf = {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20]}
grid_search_rf = GridSearchCV(rf_classifier, param_grid_rf, cv=3, scoring='accuracy')
grid_search_rf.fit(X_train_tfidf, y_binary_train)
best_rf_model = grid_search_rf.best_estimator_
binary_predictions_rf = best_rf_model.predict(X_test_tfidf)

# Binary Classification with XGBoost and GridSearchCV
xgb_classifier = XGBClassifier()
param_grid_xgb = {'n_estimators': [100, 200, 300], 'max_depth': [3, 4, 5], 'learning_rate': [0.01, 0.1, 0.2]}
grid_search_xgb = GridSearchCV(xgb_classifier, param_grid_xgb, cv=3, scoring='accuracy')
grid_search_xgb.fit(X_train_tfidf, y_binary_train)
best_xgb_model = grid_search_xgb.best_estimator_
binary_predictions_xgb = best_xgb_model.predict(X_test_tfidf)

# Evaluate binary classification model (Random Forest)
print("Binary Classification Report (Random Forest):")
print(classification_report(y_binary_test, binary_predictions_rf))

# Evaluate binary classification model (XGBoost)
print("Binary Classification Report (XGBoost):")
print(classification_report(y_binary_test, binary_predictions_xgb))

Binary Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.77      0.92      0.84       330
           1       0.89      0.71      0.79       308

    accuracy                           0.82       638
   macro avg       0.83      0.81      0.82       638
weighted avg       0.83      0.82      0.82       638

Binary Classification Report (XGBoost):
              precision    recall  f1-score   support

           0       0.80      0.85      0.83       330
           1       0.83      0.77      0.80       308

    accuracy                           0.81       638
   macro avg       0.81      0.81      0.81       638
weighted avg       0.81      0.81      0.81       638



In [27]:
def predict_sentiment(text):
    # Preprocess the input text (e.g., remove punctuation, lowercase, etc.)
    preprocessed_text = text.lower()  # You can add more pre-processing steps as needed

    # Vectorize the preprocessed text using the TF-IDF vectorizer
    text_vectorized = tfidf_vectorizer.transform([preprocessed_text])

    # Predict sentiment using all three models
    rf_prediction = best_rf_model.predict(text_vectorized)
    xgb_prediction = best_xgb_model.predict(text_vectorized)
    logistic_prediction = binary_classifier.predict(text_vectorized)

    return {
        "Random Forest Prediction": "positive" if rf_prediction[0] == 1 else "negative",
        "XGBoost Prediction": "positive" if xgb_prediction[0] == 1 else "negative",
        "Logistic Regression Prediction": "positive" if logistic_prediction[0] == 1 else "negative",
    }

# Example usage:
input_text = "telkomsel tidak jelas membuat produk, programmernya gaji buta"
sentiment_prediction = predict_sentiment(input_text)
print(sentiment_prediction)

{'Random Forest Prediction': 'negative', 'XGBoost Prediction': 'negative', 'Logistic Regression Prediction': 'negative'}


In [28]:
# Example usage:
input_text = "produknya oke, karyawannya juga ganteng dan tampan"
sentiment_prediction = predict_sentiment(input_text)
print(sentiment_prediction)

{'Random Forest Prediction': 'positive', 'XGBoost Prediction': 'positive', 'Logistic Regression Prediction': 'positive'}


## MultiClass

In [29]:
df

,contentp_clean,sentiment_first_scheme,score
0,makin sini makin parah lambat mahal,negative,1
1,makin hari paket nya makin mahal,negative,1
2,semua saran kami laku hasil tetap lambat sisa ...,negative,1
3,aplikasi baik,positive,5
4,siapa sesal beli modem aku aku aku paket nya m...,negative,1
...,...,...,...
3183,internet jelek,negative,1
3184,pilih menu beli paket nya ada ganggu,positive,5
3185,sekarang bisa beli kuota gb cuma gb gb buat ap...,negative,1
3186,pakai orbit hampir thn kemarin mau sambung hp ...,positive,5


In [45]:
df

,contentp_clean,sentiment_first_scheme,score,sentiment_second_scheme,sentiment_encoded
0,makin sini makin parah lambat mahal,negative,1,negative,0
1,makin hari paket nya makin mahal,negative,1,negative,0
2,semua saran kami laku hasil tetap lambat sisa ...,negative,1,negative,0
3,aplikasi baik,positive,5,positive,2
4,siapa sesal beli modem aku aku aku paket nya m...,negative,1,negative,0
...,...,...,...,...,...
3183,internet jelek,negative,1,negative,0
3184,pilih menu beli paket nya ada ganggu,positive,5,positive,2
3185,sekarang bisa beli kuota gb cuma gb gb buat ap...,negative,1,negative,0
3186,pakai orbit hampir thn kemarin mau sambung hp ...,positive,5,positive,2


In [30]:

df['sentiment_second_scheme'] = df['score'].apply(lambda x: 'negative' if x in [1, 2] else ('neutral' if x == 3 else 'positive'))

In [36]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['sentiment_encoded'] = label_encoder.fit_transform(df['sentiment_second_scheme'])




In [37]:
X = df['contentp_clean']  # Text data
y_multi = df['sentiment_encoded']  # Multi-class labels ("positive," "negative," "neutral")

X_train, X_test, y_multi_train, y_multi_test = train_test_split(
    X, y_multi, test_size=0.2, random_state=42
)

In [38]:
# TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Multi-class Classification with Logistic Regression
logistic_classifier = LogisticRegression(max_iter=1000)
logistic_classifier.fit(X_train_tfidf, y_multi_train)

# Predict multi-class sentiment using Logistic Regression
multi_predictions_logistic = logistic_classifier.predict(X_test_tfidf)

# Evaluate multi-class classification model (Logistic Regression)
print("Multi-class Classification Report (Logistic Regression):")
print(classification_report(y_multi_test, multi_predictions_logistic))

Multi-class Classification Report (Logistic Regression):
              precision    recall  f1-score   support

           0       0.78      0.94      0.85       330
           1       0.00      0.00      0.00        53
           2       0.86      0.80      0.83       255

    accuracy                           0.81       638
   macro avg       0.54      0.58      0.56       638
weighted avg       0.74      0.81      0.77       638



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
# Multi-class Classification with Random Forest and GridSearchCV
rf_classifier = RandomForestClassifier()
param_grid_rf = {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20]}
grid_search_rf = GridSearchCV(rf_classifier, param_grid_rf, cv=3, scoring='accuracy')
grid_search_rf.fit(X_train_tfidf, y_multi_train)
best_rf_model = grid_search_rf.best_estimator_

# Multi-class Classification with XGBoost and GridSearchCV
xgb_classifier = XGBClassifier()
param_grid_xgb = {'n_estimators': [100, 200, 300], 'max_depth': [3, 4, 5], 'learning_rate': [0.01, 0.1, 0.2]}
grid_search_xgb = GridSearchCV(xgb_classifier, param_grid_xgb, cv=3, scoring='accuracy')
grid_search_xgb.fit(X_train_tfidf, y_multi_train)
best_xgb_model = grid_search_xgb.best_estimator_



In [42]:
# Evaluate binary classification model (Random Forest)
multi_predictions_rf = best_rf_model.predict(X_test_tfidf)
multi_predictions_xgb = best_xgb_model.predict(X_test_tfidf)

print("Multi Classification Report (Random Forest):")
print(classification_report(y_multi_test, multi_predictions_rf))

# Evaluate binary classification model (XGBoost)
print("Multi Classification Report (XGBoost):")
print(classification_report(y_multi_test, multi_predictions_xgb))

Multi Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.76      0.94      0.84       330
           1       0.00      0.00      0.00        53
           2       0.86      0.78      0.82       255

    accuracy                           0.80       638
   macro avg       0.54      0.57      0.55       638
weighted avg       0.74      0.80      0.76       638

Multi Classification Report (XGBoost):
              precision    recall  f1-score   support

           0       0.78      0.89      0.83       330
           1       0.25      0.02      0.04        53
           2       0.82      0.82      0.82       255

    accuracy                           0.79       638
   macro avg       0.62      0.58      0.56       638
weighted avg       0.75      0.79      0.76       638



In [44]:
def predict_sentiment_multiclass(text):
    # Preprocess the input text (e.g., remove punctuation, lowercase, etc.)
    preprocessed_text = text.lower()  # You can add more pre-processing steps as needed

    # Vectorize the preprocessed text using the TF-IDF vectorizer
    text_vectorized = tfidf_vectorizer.transform([preprocessed_text])

    # Predict sentiment using all three models
    rf_prediction = best_rf_model.predict(text_vectorized)[0]
    xgb_prediction = best_xgb_model.predict(text_vectorized)[0]
    logistic_prediction = logistic_classifier.predict(text_vectorized)[0]

    # Map numeric labels to sentiment labels
    sentiment_labels = {0: "negative", 1: "neutral", 2: "positive"}

    return {
        "Random Forest Prediction": sentiment_labels[rf_prediction],
        "XGBoost Prediction": sentiment_labels[xgb_prediction],
        "Logistic Regression Prediction": sentiment_labels[logistic_prediction],
    }

# Example usage:
input_text = "biasa saja"
sentiment_prediction = predict_sentiment_multiclass(input_text)
print(sentiment_prediction)

{'Random Forest Prediction': 'positive', 'XGBoost Prediction': 'positive', 'Logistic Regression Prediction': 'positive'}


In [47]:
predicted_sentiments_xgb = label_encoder.inverse_transform(multi_predictions_xgb)

# Create a DataFrame to store the test data and predictions
results_df = pd.DataFrame({'Text': X_test, 'True Sentiment': label_encoder.inverse_transform(y_multi_test),
                            'XGBoost Prediction': predicted_sentiments_xgb})

In [51]:
results_df[results_df['XGBoost Prediction'] == 'neutral']

,Text,True Sentiment,XGBoost Prediction
3130,beli paket nya error,negative,neutral
2464,kalau tarif tambah mahal uninstall ah,negative,neutral
495,aplikasi sering alami crash crash alami buka m...,negative,neutral
2832,kasih bintang dulu kuota bonus nya tambah kan,neutral,neutral
